# ACSE-1 Lecture 11

## Code verification

This lecture is based on the following technical report which should be read as part of this lecture:

[Salari, Kambiz, and Patrick Knupp. Code verification by the method of manufactured solutions. No. SAND2000-1444. Sandia National Labs., Albuquerque, NM (US); Sandia National Labs., Livermore, CA (US), 2000.](https://www.osti.gov/biblio/759450)


## Learning outcomes
* You will able to define software verification and distinguish it from validation.
* Understand the different forms of code verification and how to apply them.
* Be able to apply the method of manufactured solutions (MMS) to a simple finite difference solver.

## 1. Introduction - how do you *know* your code is right (and what does right mean)?

In this lecture we learn about **verification** of computer codes that simulate physical phenomena
such as fluid dynamics, heat flow, groundwater flow, porous media transport, magnetohydrodynamics,
and structural mechanics by solving a system of partial differential equations
using some ordered discrete approximation method (e.g., finite elements, boundary elements, finite
volume, and finite difference methods).

Code Verification is important because it indicates whether or not, *the codes are solving their respective
equations correctly.* How can one systematically show that codes are correct? The fact is that there is not a universal consensus on how to do this. When you read journal articles dealing with software as a tool, and even if you work with industry, you will notice that there is a high variance in the methodology and rigour of code verification. There is not even a completely consistent definition of code verification therefore we will start by adopting the Roache definition of code verification:

> The [code] author defines precisely what continuum partial differential
> equations and continuum boundary conditions are being solved, and
> convincingly demonstrates that they are solved correctly, i.e., usually with
> some order of accuracy, and always consistently, so that as some measure of
> discretization (e.g. the mesh increments) Δ→0, the code produces a solution
> to the continuum equations; this is Verification.

From this definition we can see that if the equations are being solved correctly then the observed
discretization error decreases to zero as the mesh increments (in space and time) decrease to zero. In other words, code Verification is a procedure to demonstrate that the governing equations, as implemented in the code, are solved consistently - ie as you refine your grid (and time step) you should converge to a single solution. However, as you will see reading the accompanying paper, this criteria is not sufficiently strong for code verification and one should when possible demonstrate that the equations are solved to the
theoretical order-of-accuracy of the discretization method used.

> **NOTE:** Code verification is not concerned with asking the question, *are we solving the
> right equation?* - that is a question of **code validation**.

It is important to distinguish between the *verification of codes* and the *verification of calculations*. In the definition adopted here code verification does not concern itself with whether or not a
specific calculation using the code is accurate. Code verification need be performed only
once and from that point on, one can claim the code is solving its equations *correctly*. In contrast,
verification of calculations (also referred to as solution verification) is concerned with estimating
the overall magnitude of the discretization error of a particular calculation. Solution
verification methodology is an on-going process which should be applied to every set of
calculations. Unfortunately, both of these activities use the word verification, which creates confusion.

> **NOTE:** Code verification is a pre-requisite to solution verification.

One of the most robust methods for software verification is a formal procedure known as the Method of Manufactured solutions (MMS). In the accompanying paper, nearly a dozen codes are verified by the MMS procedure and
have developed many details of the method. The focus here is to describe the MMS
procedure by explaining in detail the art of applying MMS to Verification of engineering software.

## 2. Fundamentals of Code Verification
### 2.1 Governing Equations
Development of computer codes that simulate physical phenomena by means of ordinary
differential equations (ODEs) or partial differential equations (PDEs) is based on a sequence of
prerequisite activities. The first step is the derivation of the governing ODEs or PDEs, usually from
a set of conservation laws that represent mathematical statements of the behavior of the physical
system. For example, a code that simulates heat conduction in a stationary medium may be based
on the following equations derived from the conservation of energy:
\begin{equation}
\nabla \cdot k \nabla T + g = \rho C_p \frac{\partial T}{\partial t}
\label{eq:heat_pde}
\end{equation}
where $\rho$ is the mass density, $C_p$ is the specific heat, $T$ is the temperature, $k$ is the thermal conductivity and g is the heat generation rate (ie a source term).

Computer codes that solve Equation (1) will not all be the same because of a number of assumptions
that can be made. For example, one code might restrict itself to isotropic heat conduction, in which
the matrix $k$ reduces to a scalar. Another code might consider only steady-state heat conduction in
two dimensions while another code simulates transient heat conduction in three dimensions. A still
more general code may consider nonlinear phenomena wherein the thermal conductivity is a
function of temperature. Each of these codes would solve a slightly different set of governing
equations.

The interior equation (1) does not, by itself, define a unique solution because it lacks boundary
conditions and a statement of the domain on which the equation is to hold. Four general types of
boundary conditions are usually applied to heat conduction:
\begin{eqnarray}
\text{Dirichlet} & T &=& f (x, y, z, t)\\
\text{Neumann} & \frac{\partial T}{\partial n} &=& f (x, y, z, t)\\
\text{Robin} & k \nabla T + hT &=& f (x, y, z, t)\\
\text{Cooling and radiation} &\quad k \frac{\partial T}{\partial n} + q_{sup} &=& h(T - T_{\infty}) + \epsilon \sigma (T^4 - T^4_r)
\label{eq:heat_pde_bc}
\end{eqnarray}

where $h$ is the heat transfer coefficient, $\epsilon$ is the emissivity of the surface, and $\sigma$ is the Stefan-Boltzmann constant, $T_r$ is the effective temperature, and $q_{sub}$ is the supplied heat flux. Other types
of boundary conditions can also be posed. Some computer codes may be restricted to solving only
a subset of these boundary conditions, others will be fully general. One also needs an initial
condition,
\begin{equation}
T(x, y, z, t_0) = T_0(x, y, z)
\end{equation}

We have previously stated that to verify software that solves PDE’s by some ordered method, one
must demonstrate that the governing equations are solved consistently. To make this statement
more rigorous, we must account for the fact that there are auxiliary conditions, namely, the
boundary conditions, the initial conditions, and the domain on which the problem is formulated.

PDE’s that involve conservation statements on the domain are usually referred to as the interior
equations. Here we will refer to the interior equations plus the auxiliary conditions as the
governing equations. Thus, in our definition of code verification, solving the equations correctly
refers not only to the interior equation but to particular set (or sets) of boundary conditions, initial
conditions, and domains permitted by the code.

> **NOTE:** Code Verification entails demonstration that both the interior equations and boundary conditions are solved consistently.

### 2.2 Discretization, Consistency, and Order-of-Accuracy
Discretization of the governing equations subdivides the domain of the problem into finite cells or
elements. Rather than a continuum partial differential equation being solved for a solution in an
infinite dimensional function space, the equations are solved on a finite dimensional subspace
which approximates the continuum solution.

> **Discretization error:** The approximate solution, which satisfies the discretized equations, is not the same as the exact solution which satisfies the mathematical continuum equations. The difference between the two is called the discretization error.

> **Consistency:** Discretization methods are consistent if the error goes to zero as the representative cell size $h$ decreases to zero.

> **Order-of-Accuracy:** The rate at which the error decreases to zero is called the order-of-accuracy. For example, a discretization method is said to be second-order accurate in space if the discretization error goes to zero as $h^2$.

For brevity, we refer to a code containing a $n^{th}$-order accurate discretization method as a $n^{th}$-order accurate code. Terms in the governing equations may be discretized with different orders of accuracy. For example, it is not uncommon in an advection-diffusion equation for the advection term to be discretized to first-order accuracy while the diffusion term is second-order accurate. The over-all accuracy of such a code is the lowest order of all the approximations made (first-order in this example). A code may be first-order in time if the discretization error goes to zero as $\Delta t$ goes to zero and second-order in space. Even though two codes may solve the same mathematical equation set, they can still differ in the method of discretization. For example, the equations can be discretized by various finite difference, finite volume, finite element, or boundary element methods. One code may be second-order while another is fourth-order spatially accurate. Finally, with a discretization method chosen and some statement of the corresponding order-of accuracy, one is ready to implement the discrete algorithm in a piece of software called code.

### 2.3 Software Quality Assurance (SQA)

We briefly discuss Software Quality Assurance because some of the methods of SQA are relevant to code Verification. SQA is a formal procedure developed to ensure that software systems are reliable and secure (see paper). Current research in SQA is driven by high integrity systems such as control systems for aircraft and spacecraft, software for nuclear weapons and safety, and control systems for nuclear power reactors. The part of SQA that is most relevant to code verification is the set of procedures developed for software testing. These procedures consist of three parts: **static**, **dynamic**, and **formal testing**.

Static testing is an important prerequisite to the verification process. These tests are performed without
running the code. Obviously, we need a code that compiles and is free of compilation errors. Next, the code is checked for consistency in the usage of the computer language. This step requires additional software such as commonly used Lint or Fortran checkers. **In the case of Python for example this means using PEP8 or pylint.** These software tools are excellent in finding variables that are used but not initialized and also, in matching the argument list of a calling statement with the subroutine or the function argument list. The static tests described above are not sufficient to identify all coding and conceptual mistakes.

After completion of static testing one performs dynamic testing in which the code is run and checked for array indices that are out of bounds, for memory leakage, coverage issues and other problems that can be flagged by the compiler or other software. Code Verification can be viewed as an essential part of dynamic testing of engineering software.

After the code has been verified, remaining errors can be identified during formal analysis.

To this point we have been casual concerning our use of the word "error". To avoid confusion we
must carefully distinguish between two relevant meanings of this word. In code testing, the word
error generally refers to code mistakes (bugs) of the type already described in this Section. On the
other hand, in code verification we are concerned both with code mistakes and discretization error.

> Discretization error is the result of solving the continuum PDEs numerically and is not a code
mistake.

Discretization error is related to measures of the difference between the exact solution to the governing equations and the numerical solution to the discretized equations. In the context of code verification we will use the word "error" to refer to discretization error and the word "mistake" to refer to coding bugs or blunders.

### 2.4 A Taxonomy of Code Mistakes
For the purpose of code Verification, it is useful to define a particular taxonomy of code mistakes. Mistakes detected by the static tests described in the previous Section we call static mistakes. Mistakes detected by running the code we call dynamic mistakes. Mistakes that remain after static and dynamic tests we call formal mistakes. An example of a formal mistake would be a line of code that calculates a quantity that is not used.

Dynamic mistakes affect codes in several ways; they can affect code convergence, efficiency, and order-of-accuracy. **A coding mistake that reduces the observed order-of-accuracy will be referred to as an Order-of-Accuracy Mistake (OAM).** Order-of-accuracy mistakes can be further sub-divided into those which are severe enough to reduce the functional order-of-accuracy of the code to zeroth-order or below and those mistakes that reduce the order to a lesser degree. The former type of mistake can be referred to as a consistency mistake.

With these definitions, we can now interpret the definition of code verification as being the process of identifying and eliminating OAM's. In dynamic testing, a code can occasionally diverge. This can happen for two basic reasons:

* bad input or,
* a dynamic coding mistake.

A coding mistake that causes the code to diverge we call a divergence mistake. Efficiency mistakes are mistakes which prevent the code from attaining its theoretical performance. Other dynamic coding mistakes also exist (for example, an incorrect output format). **By definition, code verification should catch all order-of-accuracy mistakes and may also catch some coding mistakes of the other types.** To complete our taxonomy, conceptual mistakes are mental misconceptions involving the description of the algorithm that, when implemented, cause the code to not perform according to expectation. For example, a researcher devises a method for solving a non-linear equation which he believes to be quadratically convergent, but code testing reveals that the method is, in fact, only linearly convergent.

### 2.5 Methods of Dynamic Code Testing
Dynamic tests related to code verification are generally designed with two parts. The first part involves test selection and the second part test acceptance criteria. The following list provides commonly used dynamic testing approaches that have been applied by practitioners in the past. Of course, some tests are harder to pass than others. As we move from the top of the list to the bottom, we generally increase the difficulty of the test. In practice, one may chose more than a single testing approach. Only MES and MMS are appropriate for code verification.

**Dynamic Testing approaches**
* Trend
* Symmetry
* Comparison
* Exact (MES)
* Manufactured solution (MMS)

The next list gives possible acceptance criteria which might be used for a given testing approach, in order of increasing rigor.

**Acceptance Criteria**
* Expert Judgement
* Percent Error
* Consistency
* Order-of-Accuracy

#### 2.5.1 Trend Tests
In the Trend Method, a set of calculations (whose solution is not known) is performed, by varying
input parameters. Whether or not the code passes the test is determined by expert judgement, in
which the solution is examined for the right trend. For example, a heat conduction code may be run
to a steady-state solution with several values of the specific heat and, if the time it takes to reach
steady state decreases as the specific heat is decreased, then the expected trend has been
reproduced. Also, the steady-state solution is expected to be smoothly varying so, if the numerical
solution does not have this property, a coding mistake is suspected. The trouble with this approach
is that codes can readily produce plausible solutions that are quantitatively incorrect. For example,
although the trend towards the time to reach steady-state is correct, the true time to steady-state
might be incorrect by a factor of two because of order-of-accuracy mistakes in the time derivative.
The trend method sets a very low bar for a code to jump over and should be considered a minimal
test that is best used during the code development stage, not the code Verification stage. Most
unverified codes would pass the physical trend test.

#### 2.5.2 Symmetry Tests
The Symmetry Method checks for solution symmetries which can be detected with no knowledge
of the exact solution. We mention three cases. The first case is to set up a problem which, a priori,
must have a spatially symmetric solution (perhaps due to the choice of boundary conditions). For
example, the solution to a fully-developed channel flow problem can be symmetric. One inspects
the solution to see if the code generates a symmetric answer. In the second case one checks for
coordinate invariances such as translation and rotation. In this case the solution need not be
symmetric. One calculates the solution to some problem and can then do the following tests (a)
translate and rotate (say, by 90 degrees) the domain in physical space and (b) translate and rotate
the coordinate systems. The solution should remain the same as in the original problem. In the third
case, one performs a symmetric calculation using a 3-D code. The symmetry is constructed so that
the results can be compared to a 2-D solution that is known analytically. The symmetry procedures
provide a simple and effective way of testing the code for possible coding mistakes. This set of
tests is most suitable for code development.

#### 2.5.3 Comparison Tests
Another widely used approach to code testing is the Comparison Method in which one code is
compared to an established code or set of codes that solve similar problems. A test case (usually
realistic in practice) is devised and all the codes are run on the same test case. The usual acceptance
criteria is based on a computation of the maximum difference in the solutions. In practice, if the
percent difference between the solutions is within some tolerance, then the code is considered
likely to be free of coding mistakes. The main advantage of this test is that it does not require an
exact solution. Usually, calculations are performed on only a single grid. The procedure just
outlined is incomplete because grid refinement is needed to determine whether the results from the
codes are in the asymptotic range (this is seldom done in practice.)
The major difficulty with the Comparison Method centers around the fact that the codes usually
are not identical, neither in the set of equations and input parameters that they solve nor the
numerical method used. This often results in apples to oranges type comparisons wherein the
inputs to the codes have to be fudged in some way in order to make any kind of comparison.
Judgment of code correctness ends up based on ambiguous results. A related problem with the
Comparison Method is that, sometimes, no comparable code can be found. Another problem is
that, in practice, comparison tests are usually run on a single physically realistic problem, therefore
the comparison lacks generality. Reliability of the conclusions reached in the comparison test
depend heavily on the confidence that can be placed in the established code. A successful
comparison test does not rule out the possibility that both codes contain the same mistake.

#### 2.5.4 Method of Exact Solutions (MES)
A widely-used method, known as the Method of Exact Solutions (MES), is more rigorous and
objective than either the Trend or Comparison tests, In the MES code Verification procedure one
first seeks exact solutions (or a benchmark) to the set of equations solved by the code. This is
typically done by consulting the literature for published solutions. One can derive their own exact
solution, for example, using mathematical methods such as the separation of variables, or integral
transform techniques (Green’s functions, Laplace Transforms, etc.).

The exact solution is a closed-form mathematical expression that gives values of the solution at all locations in space and time. For the heat conduction problem, one finds a solution to the partial differential equation corresponding to a set of material properties, initial conditions, and boundary conditions. Having this solution, the code is then run with corresponding inputs and the numerical (discrete) solution is compared to the exact solution. If the code fails to pass the acceptance criterion, then a coding mistake is suspected. If the code passes a comprehensive suite of such tests, the code is considered adequately verified, i.e., the probability of a coding mistake is deemed small.

The MES procedure is capable of verifying codes according to the definition adopted in the introduction. However, as with all methods, there are limitations. Recall that in our definition of code verification, one must demonstrate that the governing equations are solved correctly. The governing equations are usually rather general, involving nonlinearity, coupling between equations, complex boundary conditions, variable coefficients, and geometrically complex domains. A difficulty arises at this point because methods to find exact solutions to the governing mathematical equations usually require simplifying assumptions in order to solve the equations. For example, in the Laplace transform technique one assumes that the coefficients of the PDE are
constants. If the computer code solves the heat conduction equation for heterogeneous materials
having different thermal conductivities, then a Laplace transform solution will not test this code
capability. Another common technique is to find exact solutions to one-dimensional problems
because this is more easily accomplished. A one-dimensional solution is clearly inadequate to
verify codes which simulate two- and three-dimensional heat conduction. Exact solutions often
rely on the physical domain having a simple shape such as a rectangle or disk. Many modern
computer codes can simulate physical phenomenon on very general shapes using finite element
meshes or boundary-conforming structured grids. Such a capability is not adequately tested by
comparing to exact solutions on simple domains because many of the terms in the equation become
trivial. Exact solutions are often obtained by assuming that the domain is infinite or semi-infinite.
Such domains do not fit within digital computers very well unless the code permits one to use a
coordinate transformation.

To adequately verify a modern computer code, it is necessary to have solutions which test the fully general equations solved by the code. For example, there exist in the literature exact solutions to various equations from fluid dynamics. In general, however, one can rarely find a single exact solution that solves the fully general set of governing equations. The MES work-around for this difficulty is to require a comprehensive suite of analytic solutions which, in total, comes closer to showing that the general governing equations are solved correctly. The major limitation of the MES procedure is that it is difficult, if not impossible, to create a comprehensive suite of tests that adequately exercises the fully general set of governing equations. Thus, code mistakes can still exist even after the MES procedure is performed.

A secondary limitation of MES is that exact solutions derived via Laplace Transforms, etc., can be very difficult to implement. Given an exact solution, one must compute its value at a number of points in space and time in order to compare it with the numerical solution produced by the computer code to be verified. This is often very difficult to do because the exact solutions produced by the classical mathematical techniques often involve infinite sums of terms, non-trivial integrals, and special functions (such as Bessel). To evaluate an exact solution containing an infinite series one encounters the problem of convergence and when to terminate the series. If an integral is involved, one must consider which numerical integration scheme to invoke and its accuracy. Sometimes the integrand in these exact expressions contains a singularity in the domain of integration which leads to a host of numerical difficulties. Evaluating such an integral numerically often requires consulting with an expert. As a general recommendation, developers of codes that implement analytic solutions to the governing equations should provide an assessment of the accuracy (how many significant figures) of the computed exact solution. Sometimes, this can be a challenge.

Another limitation that can arise with MES is that exact solutions may contain singularities. One cannot apply the order-of-accuracy criterion to solutions containing singularities. Even consistency can be difficult to demonstrate in such cases due to practical limitations on the number of grid points.

#### 2.5.5 Method of Manufactured Solutions (MMS)
A more comprehensive (and often easier) alternative to the method of exact solution is the Method of Manufactured Solutions (MMS). This technique is described in detail in the next section. 

#### 2.5.6 Acceptance Criteria
In each of the testing approaches there must be a test acceptance (pass-fail) criterion. The four possible criteria that could be used in these tests were the expert judgement criterion (used in the Trend and Symmetry tests), the percent difference/error criterion (used mainly in the comparison method), the consistency criterion, and the order-of-accuracy criterion. Recall that a discretization of a PDE is consistent if, as the mesh size decreases to zero, so does the discretization error. If the mesh size is measured by the parameter $h$, then a consistent method will result in error that is proportional to $h^p$, where $p>0$. In code testing this means that the code should reduce the error to within machine roundoff with a sufficiently fine mesh. In practice this criterion, which requires calculations of the solution on several mesh sizes, is seldom used. The most rigorous acceptance criterion is verification of order-of-accuracy, in which one not only seeks to verify that the method is consistent, but also establishes the value of $p$, which is then compared to the theoretical order of
the discretization method. For each testing method one may choose among these four acceptance criteria. For example, MES could be associated with either the percent error, consistency, or the order-of-accuracy criteria. Thus there are at least three possible varieties of MES, with the order-of-accuracy criterion being the most rigorous. In the same way, the method of manufactured solutions (MMS) can be associated with one of these three acceptance criteria. In the following description of MMS we advocate that MMS be associated with the order-of-accuracy criterion, resulting in the most comprehensive and rigorous of all code Verification methods. Table 1 shows the possible associations between test methods and acceptance criteria. “Yes” signifies that the association is applicable. The Trend and Symmetry methods focus mainly on physical trends and do not involve mesh refinement, thus the last three acceptance criteria are not usually applied. The comparison method compares solutions from two or more codes. Typically, only the first two acceptance criteria are applied. Both MES and MMS use exact solutions so the last three acceptance criteria can theoretically be applied. In practice MES often uses the percent error criterion but can be made more rigorous by adopting the consistency or order-of-accuracy criteria. MMS can rigorously be applied using the order-of-accuracy criterion.

Table 1: Possible associations between code testing methods and acceptance criteria.

|           |Expert Judgement | Percent Error | Consistency | Order-of-Accuracy |
|-----------|-----------------|---------------|-------------|-------------------|
|Trend      | Yes             | No            | No          | No                |
|Symmetry   | Yes             | No            | No          | No                |
|Comparison | Yes             | Yes           | No          | No                |
|MES        | No              | Yes           | Yes         | Yes               |
|MMS        | No              | Yes           | Yes         | Yes               |

### 3 Method of Manufactured Solution (MMS)
A manufactured solution is an exact solution to some PDE or set of PDE’s that has been
constructed by solving the problem backwards. Suppose one is solving a differential equation of
the form

\begin{equation}
\pmb{D}\pmb{u} = \pmb{g}
\end{equation}

where $\pmb{D}$ is the differential operator, $\pmb{u}$ is the solution, and $\pmb{g}$ is a source term. In the method of exact solution (MES), one chooses the function $\pmb{g}$ and then, using methods from classical applied
mathematics, inverts the operator to solve for $\pmb{u}$. In MMS, one first manufactures a solution $\pmb{u}$ and
then applies $\pmb{D}$ to $\pmb{u}$ to find $\pmb{g}$ (a much simpler procedure). In the MMS code verification procedure, one manufactures a solution to the fully general set of interior equations solved by the code that is
to be verified. The reason a solution to the general equations is wanted is to test all portions of the code.

Lets apply the manufactured solution technique to solve the heat conduction equation (1). The strength of this approach is that we can create a general solution that exercises all the terms and the coefficients of the interior equation without imposing special coordinate systems or particular boundary conditions.

The following is a manufactured solution and associated coefficient functions that we designed for the heat equation.

\begin{eqnarray}
T(x, y, z, t) = T_0\left[1 + \sin^2 \left( \frac{ x}{R} \right)
                                \sin^2 \left( \frac{2y}{R} \right)
                                \sin^2 \left( \frac{3z}{R} \right) \right]    
                                e^{t(t_0-t)/t_0}
\end{eqnarray}

\begin{eqnarray}
k(x, y, z) &=& k_0\left(1 + \frac{\sqrt{x^2 + 2y^2 + 3z^2}}{R}\right)\\
\rho(x, y, z) &=& \rho_0\left(1 + \frac{\sqrt{3x^2 + y^2 + 2z^2}}{R}\right)\\
C_p(x, y, z) &=& C_{p0}\left(1 + \frac{\sqrt{2x^2 + 3y^2 + z^2}}{R}\right)
\end{eqnarray}

where $k_0$, $r_0$, $C_{p0}$, $t_0$, $T_0$, and $R$ are constants which determine the units and range of each function. The source term for the manufactured solution is computed from
\begin{equation}
g(x, y, z, t) = \rho C_p \frac{\partial T}{\partial t} - \nabla \cdot k \nabla T
\end{equation}

## Exercise 1.3.1
> In the original paper the authors used the symbolic manipulation code Mathematica to substitute expressions for $T$, $k$, $\rho$ and $C_p$ into the source term $g$.
>
> [SymPy](https://www.sympy.org/en/index.html) is a Python library for symbolic mathematics. It aims to become a full-featured computer algebra system (CAS) while keeping the code as simple as possible in order to be comprehensible and easily extensible. SymPy is open source and written entirely in Python.
>
> * Use SymPy to form symbolic expressions for $T$, $k$, $\rho$ and $C_p$.
> * Use these symbolic expressions to create a symbolic expression for the source, $g$.

Usually, the source term created by the method of manufactured solutions is a distributed source, i.e., not a point source. This can give rise to a difficulty in the case that the code to be verified does not allow distributed sources. A related difficulty is that the code may not contain a source term at all.

The nice thing about SymPy (and similar symbolic manipulation codes) is that it can create FORTRAN or C statements of the source term. From that one can create an auxiliary code to evaluate the source term at the required locations and time.

There are many other possible manufactured solutions to Equation 1 besides the one given. The analytic solution to Equation 1 obtained by MMS is very general (being heterogeneous and non-steady) and exercises all the terms and coefficients in the governing equation. There is no difficulty computing the solution accurately since, by design, the solution consists of simple (primitive) functions that can be evaluated to virtually machine accuracy.

We have not, to this point, discussed how the domain, the initial condition, and boundary
conditions are handled by MMS. These will be discussed in Section 3.3, but first we cover some guidelines for the construction of the solution to the interior equations.

### 3.1 Guidelines for Creating Manufactured Solutions
When constructing manufactured solutions, we recommend that the following guidelines be
observed. The guidelines are needed to ensure that the numerical solution can be computed
accurately and with little difficulty. It is usually not hard to create solutions within these guidelines.

* First, manufactured solutions should be composed of smooth analytic functions like polynomials, trigonometric, or exponential functions so that the solution is conveniently computed (e.g., no infinite series). Solution smoothness is essential to ensure that the theoretical order-of-accuracy can be attained.

* Second, the solution should be general enough that it exercises every term in the governing equation. For example, do not choose temperature T to be independent of time.

* Third, the solution should have a sufficient number of non-trivial derivatives. For example, if the code that solves the heat conduction equation is second-order in space, picking T in Equation 5 to be a linear function of time and space will not provide a sufficient test because second-order accurate convergence rates would be assured.

* Fourth, solution derivatives should be bounded by a small constant. This ensures that the solution is not a strongly varying function of space and/or time. If this guideline is not met, then one may not be able to demonstrate the required asymptotic order-of-accuracy using practical grid sizes. Usually, the free constants that are part of the manufactured solution can be selected to meet this guideline. Finally, this guideline means that the solution should not contain singularities within the domain.

* Fifth, the manufactured solution should not prevent the code from running successfully to completion during testing. Robustness issues are not a part of code Verification. For example, if the code assumes the solution to be positive, then make sure the manufactured solution is positive. Or, if time is expected to be positive, make sure the solution is defined for t > 0. Or, if the heat conduction code expects time units of seconds, do not give it a solution whose units are nanoseconds.

* Sixth, the solution should be defined on a connected subset of two- or three-dimensional space. This allows flexibility in selecting the domain of the PDE (see Section 3.3.1)

* Seventh, the solution should be constructed in a manner such that the differential operators in the PDE’s make sense. For example, in the heat conduction equation, the flux is required to be differentiable. Therefore, if one desired to test the code for the case of discontinuous thermal conductivity, then the manufactured solution for temperature must be constructed in such a way that the flux is differentiable (see [7] for an example of where this was done).

The reader should realize that there is no requirement for physical realism in the manufactured solution. We maintain that such a requirement is not necessary and, in fact, is detrimental because it is often more difficult to construct a general test problem. The code cannot make a distinction between a physically realistic problem and a non-physical one since it is only solving a set of equations.

### 3.2 Guidelines for Construction of the PDE Coefficients
In constructing a manufactured solution, we are free to choose the PDE coefficient functions, with mild restrictions similar to those imposed upon the solution function. 

* First, coefficient functions should be composed of analytic functions like polynomials, trigonometric, or exponential functions so that they are conveniently computed (e.g., no infinite series).

* Second, the coefficient functions should be non-trivial and fully general. In the heat conduction equation, for example, one should not choose the conductivity tensor to be a single constant scalar. Such a choice would fail to exercise the full code capabilities. To test the full capabilities one should choose the conductivity to be a full tensor. Well-designed codes will permit one to have spatial discontinuities in the conductivity functions and so these should be included as well.

* Third, the coefficient functions (which usually represent material properties) should be somewhat physically reasonable. For example, although one can construct solutions to the PDE in which the specific heat is negative, this violates conservation of energy and is likely to cause the code to fail over some robustness issue. Another example: the conductivity tensor is mathematically required to be symmetric, positive definite in order for the equation to be of elliptic type. If this condition is violated in the construction of the manufactured solution, one again would likely run afoul of some numerical robustness issue present in the code (e.g., if an iterative solver were used, the symmetric, positive-definite assumption is likely made in the design of the iterative scheme). One does not need to go over-board on physical realism, however; even though there is no material whose conductivity varies like a Sine function, this is often a useful choice.

* Fourth, the coefficient functions need to be sufficiently differentiable so that the differential operator make sense. For example, one should not manufacture coefficient functions which contain a singularity within the domain.

* Fifth, the coefficient functions should be within the code’s range of applicability. If the code expects heat conductivities to be no smaller than that of cork, then don’t give it a problem with near-zero conductivity. Again, the object in Verification is not to test code robustness.

### 3.3 Auxiliary Conditions
To this point we have shown how to manufacture a general solution to the interior equations and given guidelines for constructing the solution and the PDE coefficients. As noted in Section 2.1, the governing equations are not complete until the auxiliary conditions (initial condition, boundary conditions, and problem domain) are determined. An essential difference between the MES and MMS code Verification procedures lies in the treatment of the auxiliary conditions. With MES, the solution to the governing equations is derived by considering the interior equations and auxiliary conditions simultaneously. In MMS, the auxiliary conditions can often be considered after the solution to the interior equation has been manufactured. This flexibility is one of the most attractive
features of MMS. In this Section we discuss how the auxiliary conditions are determined for code verification by MMS.

To illustrate this difference between the two code Verification procedures, we begin with the initial condition, which is relatively easy to understand. In MES, initial conditions are given and one attempts to find an exact solution to the equation that satisfies the governing equations (including the initial condition). In the MMS procedure the initial condition poses no difficulty because the solution to the interior equation is already known. Once the manufactured solution $u(x,y,z,t)$ has been constructed, the initial condition $u_0$ is simply found by evaluating the manufactured solution $u$ at $t=t_0$, i.e., $u_0(x,y,z) = u(x,y,z,t_0)$. For example, in our manufactured solution to the heat conduction equation, Equation 5 yields
\begin{equation}
T(x, y, z, t) = T_0\left[1 + \sin^2 \left( \frac{ x}{R} \right)
                                \sin^2 \left( \frac{2y}{R} \right)
                                \sin^2 \left( \frac{3z}{R} \right) \right]
\end{equation}
for the initial condition.

In the MMS code Verification procedure, this function is evaluated at discrete locations dictated by the mesh and the values of the function are given to the code as input. Because the manufactured solution solves the fully general equations, the input initial
condition derived from the manufactured solution is also general (e.g., spatially varying). To achieve flexibility, engineering codes usually solve more than one set of auxiliary conditions. For example, the code may provide options for various boundary condition types (e.g., slip, noslip, inflow, outflow, symmetry, and periodic). The code is usually flexible concerning the location on the domain boundary where each type is to be imposed. A number of code Verification issues result from this flexibility which are discussed in the subsections to follow.

#### 3.3.1 Choosing the Problem Domain
Because the manufactured solution and the PDE coefficient functions are defined on some connected subset of 2-D or 3-D space, one has considerable freedom in the selection of the problem domain. Limitations on the domain type may be imposed by the particular code being verified. For example, the code may only allow rectangular domains. Because we wish to test the code at its highest level of generality, we choose the domain as general as possible. If the most general domain the code is capable of handling is a rectangle of sides a and b, then do not choose a square domain, choose instead the rectangle. If the code can handle general, simply-connected 2-D regions, do not use a rectangle for the domain. If the code handles non-simply connected domains, and this option is to be Verified, then choose the domain accordingly. If the code is 3-D, use a 3-D domain. If the
most general domain type is used then one is more likely to find coding mistakes than if only special cases are tested. An attractive feature of the MMS procedure is that the domain can often be selected after construction of a manufactured solution.
With the MMS procedure, the process of selecting the problem domain is often, but not always, independent of constructing the boundary condition input to the code. Some exceptions are considered in the next subsection, on boundary conditions.
* 1 Strictly speaking, it is recommended that one not use the initial condition derived from the exact solution. Although, this would minimize the number of iterations needed to converge the solution, the danger of this approach is that it can hide coding mistakes. We recommend that one use the initial condition derived from the manufactured solution multiplied by some constant not too close to one.
* 2 For example, Equation 5 shows that any subset of $R^3$ is a potential domain.

#### 3.3.2 Boundary Conditions
Treatment of boundary conditions is one of the most difficult topics in code verification. Boundary conditions generally possess three important attributes: type, location, and value. As mentioned earlier, there are many different types of boundary conditions such as Dirichlet, Neumann, and Mixed. The second attribute of boundary conditions is location, i.e., the portion of the domain boundary on which each boundary condition type is imposed. For example, if the domain is a rectangle, perhaps the top and bottom boundaries are flux while the left and right are Dirichlet. The third attribute is value, i.e., the numerical value imposed by the boundary condition at a given location. For example, the flux at a point may be zero (a homogeneous flux condition) or non-zero (an in homogeneous flux condition). Either way, the numerical value is one of the required code inputs for this boundary condition type. Another important observation concerning boundary conditions is that one needs boundary conditions for each dependent variable in the PDE.

Assume for the moment that both the type and location of the boundary condition has been given. Then, in the MMS procedure the value attribute can be computed from the manufactured solution. A very attractive feature of MMS, then, is that boundary condition type and location can often be selected after construction of the manufactured solution. We illustrate this point for various boundary condition types.

*Dirichlet Boundary Conditions.* If a Dirichlet condition is imposed on a given dependent variable for some portion of the domain boundary, then the value attribute is easily computed by evaluating the manufactured solution to the interior equations at the relevant points in space dictated by the location attribute and the grid on the boundary. The calculated values form part of the code input, along with the boundary condition type and location. If the manufactured solution is timedependent, then the value attribute must be calculated for each time step.

In the heat conduction example, the value of the Dirichlet condition on the boundary $x=L_x$ is
\begin{eqnarray}
T(x, y, z, t) = T_0\left[1 + \sin^2 \left( \frac{ x}{R} \right)
                                \sin^2 \left( \frac{2y}{R} \right)
                                \sin^2 \left( \frac{3z}{R} \right) \right]    
                                e^{t(t_0-t)/t_0}
\end{eqnarray}
Note that the boundary condition is time-dependent, as well as spatially dependent.

*Neumann Boundary Conditions.* Also known as flux boundary conditions, these take the form
\begin{equation}
K \nabla T \cdot \hat{n} = q
\end{equation}

The value attribute for this boundary condition is the numerical values of the scalar q at each point in time and space. These values can be calculated from the manufactured solution by analytically calculating its gradient (perhaps with a symbolic manipulation code) and evaluating the analytic expression at the required points in space and time. The results are used as code input.

*Cooling and Radiation Boundary Condition* The fourth boundary condition in Equation 2 can be applied on the $x=L_x$ boundary by re-writing the condition as
\begin{equation}
q_{sup} = h(T - T_{\infty}) + \epsilon \sigma (T^4 - T^4_r) + k \frac{\partial T}{\partial n}
\end{equation}

The left hand side can be evaluated using the manufactured solution and values of $k$, $h$, $\epsilon$, $\sigma$, $T_{\infty}$, and $T_r$. Then, the result is provided to the code as an input for $q_{sup}$ which in this case,
is time and spatially varying. If the code input only allows $q_{sup}$ to be constant, then a code
modification is required. If one cannot modify the code, one may be forced to leave this particular
code option unverified. This boundary condition provides an example which shows that code
modification may be necessary to verify some boundary condition options by the MMS procedure.

*Free Slip Boundary Conditions.* This condition is a Dirichlet boundary condition. The component
of velocity normal to the boundary is zero (the value attribute is zero); the component of the
velocity parallel to the boundary is not prescribed. This boundary condition type requires that the
manufactured solution have zero normal velocity at the given set of locations at which the
condition is imposed. In this case the domain is not entirely independent of the boundary
conditions. Never-the-less, it is quite possible to construct manufactured solutions adhering to this
boundary condition. The trick to satisfying this boundary condition is to manufacture a solution to
the interior equations for which the normal velocity equals zero on some curve. This curve then
determines the part of the domain boundary on which the free-slip condition will be applied. A
similar case can be found in [7], in which a zero-flux condition was enforced along a boundary
curve.

## 4 Evaluation of Discretization Error and Order-of-Accuracy
### 4.1 Discretization Error
The numerical solution consists of values of the dependent variables on some set of discrete
locations determined by the grid and discrete time levels. To compute the discretization error,
several measures are possible. Let x be a point in $R^n$ and $dx$ the local volume. To compare functions
$u$ and $v$ on $R^n$ the $L_2$ norm of $u-v$ is
\begin{eqnarray}
\|u-v\|_2 $=$ \sqrt{\int(u-v)^2 dx}\\
          $=$ \sqrt{\int(u-v)^2 J d\xi}
\end{eqnarray}
where $J$ is the Jacobian of the local transformation and $d\xi$ is the local volume of the logical space.
By analogy, for discrete functions $U$ and $V$ the $l_2$ norm of $U-V$ is
\begin{equation}
\|U-V\|_2 $=$ \sqrt{\sum_n(U_n-V_n)^2 \alpha_n}
\end{equation}
where $\alpha_n$ is some local volume measure and n is the index of the discrete solution location.

Because the manufactured solution is defined on the continuum, one can evaluate the exact (manufactured) solution at the same locations in time and space as the discrete solution. The local discretization error at point $n$ of the grid is given by $u_n - U_n$, where $u_n$ is the manufactured solution evaluated at $x_n$, $y_n$, $z_n$ and $U_n$ is the discrete solution. The normalized global error is defined by:
\begin{equation}
e_2 = \sqrt{\frac{\sum_n(u_n-U_n)^2 \alpha_n}{\sum_n \alpha_n}}
\end{equation}

If the local volume measure is constant (e.g., as in a uniform grid), the normalized global error
(sometimes referred to as the RMS error) reduces to
\begin{equation}
e_2 = \sqrt{\frac{1}{N}\sum_n(u_n-U_n)^2}
\end{equation}

From the last two equations one sees that if $u_n - U_n = \vartheta(h^p)$, then the normalized global error is $\vartheta(h^p)$. This fact enables one to ignore non-uniform grid spacing when calculating the discretization error for the purpose of code Verification (recall that one only needs to show consistency or verify the theoretical order-of-accuracy; the actual magnitude of the error is irrelevant). Thus, either of these two equations may be used in code verification exercises.

The infinity norm is another useful norm of the global error, which is defined by
\begin{equation}
e_{\infty} = \max_n |u_n - U_n|
\end{equation}

In general, both error measures should include points on the boundary.

Often engineering software computes not only the solution but certain derived variables such as
flux or aerodynamic coefficients (lift, drag, moment). For thorough Verification of the code, one
should also compute the error in these derived variables if they are provided as code output. For
example, Verification of the calculated flux (which involves the gradient of u), can use the
following global error measure

\begin{equation}
e_2 = \sqrt{\frac{1}{N}\sum_n(u'_n-U'_n)^2}
\end{equation}

The gradient of the exact solution is easily calculated (perhaps with SymPy) from the analytic expression for the exact solution. is obtained from the code output. It should be noted that the theoretical order-of-accuracy of calculated fluxes may be less than the theoretical orderof-accuracy of the interior equations. An important point is that MMS procedure is not limited to any particular error measurement. For example, one can use either the RMS the maximum norm, or both.

### 4.2 Determination of Observed Order-of-Accuracy
Given that we can obtain some measure of the global error on a grid, we then run the code on a series of different mesh sizes h (at least 2) and compute the error for each mesh. From this information, the observed order-of-accuracy can be estimated and compared to the theoretical order-of-accuracy.

For example, if the discrete solution is spatially second-order accurate and in the asymptotic regime, the error (in the RMS or infinity norms) will decrease by a factor of 4 for every halving of the grid cell size. Often the spatial error is separated out from the temporal error.

The definition of the theoretical order-of-accuracy is related to the discretization error and is a
function of $h$, where $h$ is the grid spacing:
$$E = E(h)$$

For consistent methods, the discretization error $E$ in the solution is proportional to $h_p$ where $p>0$
is the theoretical order of the method
$$E = C h^p + H.O.T.$$
where $C$ is a constant independent of $h$ and $H.O.T.$ refers to higher order terms. Because we have considerable flexibility in constructing the manufactured solution, it is not difficult to ensure that it has sufficient derivatives so the Taylor series for the manufactured solution exists. Therefore, this description of error applies to every consistent discretization method, such as finite difference method (FDM), finite volume methods (FVM), and finite element methods (FEM). It is important that the numerical procedure be consistent, which means the continuum equations are recovered as $h$ goes to zero.

To estimate the order-of-accuracy of the code being Verified we rely on the assumption that as $h$ decreases to zero, the first term in Equation 21 dominates. The global errors, $E_{grid1}$ and $E_{grid2}$, are obtained from the numerical solutions to the Verification test problems. If $grid1$ is of size $h$ and $grid2$ is of size $h/r$, where $r$ is the refinement ratio, the error has the form:
$$ E_{grid1} \approx C h^p$$
$$ E_{grid2} \approx C \left( \frac{h}{r} \right)^p$$

The ratio of the errors is given by
$$ \frac{E_{grid1}}{E_{grid2}} \approx \left( \frac{C h^p}{C h^p} \right) r^p$$

Thus, the observed order $p$ is computed as
$$ p \approx \log{\frac{E_{grid1}}{E_{grid2}}}/log(r)$$

Systematic grid refinement with a constant grid refinement ratio r produces a sequence of observed orders-of-accuracy. The trend in the sequence is then compared to the theoretical order-of-accuracy of the discretization method. Typically one should not expect to recover the theoretical order-of-accuracy to more than two or three significant figures with this procedure because in practice reducing $h$ to zero requires high precision and large numbers of elements which, in turn, result in excessive memory requirements and run-times.

To compute the observed time-order-of-accuracy one can use the same formulation presented above by replacing the grid size $h$ with the time step size $\Delta t$.

## Exercise 1.3.2
> * Implement a 1-D second order finite difference solver for the heat conduction equation with support for Dirichlet Boundary Conditions.
> * Use MMS to verify the correctness of your solver.